In [1]:
import sqlite3
import numpy as np
import pandas as pd


In [15]:

conn = sqlite3.connect('rpg_db.sqlite3') # Open connection to local DB
cursor = conn.cursor() # Reference to cursor

#conn.commit
#conn.close

In [16]:
def df_sql(query, conn=conn):
    return pd.read_sql_query(query, conn) #Function to read query from connection

In [56]:
characters = df_sql("""SELECT * FROM charactercreator_character""",conn) # run query using funtion

In [34]:
def sql_fetch(con):
 
    cursorObj = con.cursor()
 
    cursorObj.execute('SELECT name from sqlite_master') # extract list names of tables  
    
    return cursorObj.fetchall()


list_tables = [s[0] for s in sql_fetch(conn)] # iterate trhough list in orde to get last list

In [52]:
sql_fetch(conn)[0]

('django_migrations',)

In [35]:
list_tables

['django_migrations',
 'sqlite_sequence',
 'auth_group',
 'sqlite_autoindex_auth_group_1',
 'auth_group_permissions',
 'auth_user_groups',
 'auth_user_user_permissions',
 'auth_group_permissions_group_id_permission_id_0cd325b0_uniq',
 'auth_group_permissions_group_id_b120cbf9',
 'auth_group_permissions_permission_id_84c5c92e',
 'auth_user_groups_user_id_group_id_94350c0c_uniq',
 'auth_user_groups_user_id_6a12ed8b',
 'auth_user_groups_group_id_97559544',
 'auth_user_user_permissions_user_id_permission_id_14a6b632_uniq',
 'auth_user_user_permissions_user_id_a95ead1b',
 'auth_user_user_permissions_permission_id_1fbb5f2c',
 'django_admin_log',
 'django_admin_log_content_type_id_c4bce8eb',
 'django_admin_log_user_id_c564eba6',
 'armory_item',
 'armory_weapon',
 'django_content_type',
 'django_content_type_app_label_model_76bd3d3b_uniq',
 'auth_permission',
 'auth_permission_content_type_id_codename_01ab375a_uniq',
 'auth_permission_content_type_id_2f476e4b',
 'auth_user',
 'sqlite_autoindex

In [43]:
#print(cursor.execute('''SELECT name FROM charactercreator_character'''))

AttributeError: 'sqlite3.Cursor' object has no attribute 'name'

In [48]:
total_characters = len(set(df_sql('''SELECT name FROM charactercreator_character''')['name'])) # get unique characters

In [49]:
total_characters

297

In [50]:
set(df_sql('''SELECT name FROM charactercreator_character'''))

{'name'}

In [63]:
total_items = len(set(set(df_sql('''SELECT item_id FROM armory_item''')['item_id'])))

In [66]:
armory_items = df_sql('''SELECT * FROM armory_item''')
armory_items.head()

,item_id,name,value,weight
0,1,"Libero facere dolore, as",0,0
1,2,Qui,0,0
2,3,Laborios,0,0
3,4,Quibusdam illo deserunt ea,0,0
4,5,Quod eveniet i,0,0


In [64]:
total_items

174

In [91]:
# total items that are weapons
df_sql('''SELECT COUNT(item_ptr_id)
FROM   armory_weapon 
WHERE item_ptr_id IN (SELECT item_id FROM armory_item)''')


,COUNT(item_ptr_id)
0,37


In [94]:
# items that are not weapons
df_sql('''SELECT COUNT(item_id)
FROM   armory_item 
WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon)''')

,COUNT(item_id)
0,137


In [109]:
# Use COUNT() and GROUP BY in order to check how many items per charcter and using LIMIT to show first 20
items_per_character = df_sql("""SELECT COUNT(item_id), character_id 
FROM charactercreator_character_inventory 
GROUP BY character_id LIMIT 20""",conn)

In [111]:
items_per_character

,COUNT(item_id),character_id
0,3,1
1,3,2
2,2,3
3,4,4
4,4,5
5,1,6
6,5,7
7,3,8
8,4,9
9,4,10


In [208]:
# Select columns from inventory
# use count to specify count what
# LEFT JOIN weapon table ON id = id LEFT so i can have even thosetha are 0
# Group by character_id to have total number of items that are weapon
weapons_per_character = df_sql("""SELECT character_id, COUNT(item_ptr_id)
FROM   charactercreator_character_inventory
LEFT JOIN armory_weapon ON armory_weapon.item_ptr_id=charactercreator_character_inventory.item_id
GROUP BY character_id
LIMIT 20""",conn)

In [209]:
weapons_per_character

,character_id,COUNT(item_ptr_id)
0,1,0
1,2,0
2,3,0
3,4,0
4,5,2
5,6,0
6,7,1
7,8,0
8,9,0
9,10,0


In [210]:
avg_weapon_per_character = df_sql("""SELECT AVG(item_id) FROM (SELECT character_id, COUNT(item_ptr_id) as item_id
FROM   charactercreator_character_inventory
LEFT JOIN armory_weapon ON charactercreator_character_inventory.item_id=armory_weapon.item_ptr_id
GROUP BY character_id)""",conn)

In [211]:
avg_weapon_per_character

,AVG(item_id)
0,0.672185


In [212]:
avg_item_per_character = df_sql("""SELECT AVG(items) FROM (SELECT COUNT(item_id) as items, character_id 
FROM charactercreator_character_inventory 
GROUP BY character_id)""",conn)

In [213]:
avg_item_per_character

,AVG(items)
0,2.97351


In [217]:
character_inventory = df_sql("""SELECT * FROM charactercreator_character_inventory""",conn)

In [218]:
#conn.commit
conn.close

<function Connection.close>

In [214]:
buddy = pd.read_csv('buddymove_holidayiq.csv')

In [215]:
buddy.shape

(249, 7)

In [216]:
buddy.isnull().sum()

User Id      0
Sports       0
Religious    0
Nature       0
Theatre      0
Shopping     0
Picnic       0
dtype: int64

In [219]:
conn = sqlite3.connect('buddymove_holidayiq.sqlite3') # Open connection to local DB
cursor = conn.cursor()

In [222]:
buddy.to_sql('buddymove_holidayiq', conn)

C:\Users\mhdal\Anaconda3\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [226]:
n_rows = cursor.execute('''SELECT COUNT(*) FROM buddymove_holidayiq''').fetchall()

In [228]:
print(n_rows[0][0])

249


In [229]:
def df_sql(query, conn=conn):
    return pd.read_sql_query(query, conn)

In [236]:
df_sql('''SELECT COUNT(*) FROM buddymove_holidayiq''')['COUNT(*)'][0]

249

In [244]:
n_nature_shopping = cursor.execute('''SELECT COUNT(*) FROM buddymove_holidayiq WHERE Shopping>100 AND Nature>100''').fetchall()

In [246]:
n_nature_shopping[0][0]

78

In [297]:
columns = list(i[0] for i in cursor.execute('''SELECT * FROM buddymove_holidayiq''').description)
columns.remove('index')
columns.remove('User Id')

In [298]:
columns

['Sports', 'Religious', 'Nature', 'Theatre', 'Shopping', 'Picnic']

In [303]:
for i in columns:
    averages = cursor.execute("SELECT AVG("+i+") FROM buddymove_holidayiq").fetchall()
    print('Average for {} is {}'.format(i, averages[0][0]))

Average for Sports is 11.987951807228916
Average for Religious is 109.77911646586345
Average for Nature is 124.51807228915662
Average for Theatre is 116.37751004016064
Average for Shopping is 112.63855421686748
Average for Picnic is 120.40160642570281


In [290]:
cursor.execute("SELECT AVG(Sports) FROM buddymove_holidayiq").fetchall()

[(11.987951807228916,)]